In [1]:
import numpy as np
import pydae.build as db
import sympy as sym
from pydae.models import grid2dae_dq

In [9]:
U_b = 20e3
V_b = U_b/np.sqrt(3)

R_ph_km   = 0.501
X_ph_km   = 0.716
B_ph_muS_km = 47.493

Length_12_13_km = 4.89
Length_13_14_km = 2.99

R_12_13 = Length_12_13_km*R_ph_km
X_12_13 = Length_12_13_km*X_ph_km
B_12_13 = Length_12_13_km*B_ph_muS_km*1e-6

R_13_14 = Length_13_14_km*R_ph_km
X_13_14 = Length_13_14_km*X_ph_km
B_13_14 = Length_13_14_km*B_ph_muS_km*1e-6


data = {
"system":{"f_hz":50,"model_type":"ode"},
"buses": [{"bus":"12"},{"bus":"13"},{"bus":"14"}],
"lines":     [  
              # {"bus_j":"0", "bus_k":"12", "R":0.016, "X":1.92, "B":B_12_13/10},
                {"bus_j":"12", "bus_k":"13", "R_km":0.501, "X_km":0.716, "B_muS_km":47.493, "km":4.89},
                {"bus_j":"13", "bus_k":"14", "R_km":0.501, "X_km":0.716, "B_muS_km":47.493, "km":2.99},
             ],
"loads" : [
        #{"bus": "12", "kVA":0e3, "pf": 0.95, "T_i":0.01,"I_max":1200},
        #{"bus": "13", "kVA":0e3, "pf": 0.95, "T_i":0.01,"I_max":1200},
        #{"bus": "14", "kVA":0e3, "pf": 0.95, "T_i":0.01,"I_max":1200}
        ],
"grid_formers" : [
                  #{"bus": "12","V_phph":20.0e3, "deg":0.0},
                 ]}

In [10]:
15300

15300

In [11]:
grid_dae  = grid2dae_dq(data,park_type = 'fisix')

'''
grid_formers = data["grid_formers"]
for grid_former in grid_formers:
    bus = grid_former['bus']
    idx_D = grid_dae['x_list'].index(f'v_{bus}_D')
    grid_dae['f'].pop(idx_D)
    grid_dae['x'].pop(idx_D)
    grid_dae['x_list'].pop(idx_D)
    grid_dae['u'].update({f'v_{bus}_D':grid_former["V_phph"]*np.sqrt(2/3)*np.sin(np.deg2rad(grid_former["deg"]))})
    idx_Q = grid_dae['x_list'].index(f'v_{bus}_Q')
    grid_dae['f'].pop(idx_Q)
    grid_dae['x'].pop(idx_Q)
    grid_dae['x_list'].pop(idx_Q)
    grid_dae['u'].update({f'v_{bus}_Q':grid_former["V_phph"]*np.sqrt(2/3)*np.cos(np.deg2rad(grid_former["deg"]))})
'''    
f_list = grid_dae['f']  
x_list = grid_dae['x']   

g_list = grid_dae['g']  
y_list = grid_dae['y']  
params_dict = grid_dae['params'] 
u_run_dict = grid_dae['u'] 
u_run_dict.update({'ina':0})
a = sym.Symbol('a')
ina = sym.Symbol('ina')
y_list = ['a']
g_list = [1-a]
sys = {'name':'grid_3bus',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_run_dict,
       'h_dict':{'i_R06_d':y_list[0]}
      }

sys = db.system(sys)
db.sys2num(sys)

In [12]:
grid_dae

{'f': [(-L_1213*i_l_1213_Q*omega - R_1213*i_l_1213_D + v_12_D - v_13_D)/L_1213,
  (L_1213*i_l_1213_D*omega - R_1213*i_l_1213_Q + v_12_Q - v_13_Q)/L_1213,
  (-L_1314*i_l_1314_Q*omega - R_1314*i_l_1314_D + v_13_D - v_14_D)/L_1314,
  (L_1314*i_l_1314_D*omega - R_1314*i_l_1314_Q + v_13_Q - v_14_Q)/L_1314,
  2*(-C_1213*omega*v_12_Q/2 + i_12_D - i_l_1213_D)/C_1213,
  2*(C_1213*omega*v_12_D/2 + i_12_Q - i_l_1213_Q)/C_1213,
  (i_13_D + i_l_1213_D - i_l_1314_D + omega*v_13_Q*(-C_1213/2 - C_1314/2))/(C_1213/2 + C_1314/2),
  (i_13_Q + i_l_1213_Q - i_l_1314_Q - omega*v_13_D*(-C_1213/2 - C_1314/2))/(C_1213/2 + C_1314/2),
  2*(-C_1314*omega*v_14_Q/2 + i_14_D + i_l_1314_D)/C_1314,
  2*(C_1314*omega*v_14_D/2 + i_14_Q + i_l_1314_Q)/C_1314],
 'g': [],
 'x': [i_l_1213_D,
  i_l_1213_Q,
  i_l_1314_D,
  i_l_1314_Q,
  v_12_D,
  v_12_Q,
  v_13_D,
  v_13_Q,
  v_14_D,
  v_14_Q],
 'y': [],
 'x_list': ['i_l_1213_D',
  'i_l_1213_Q',
  'i_l_1314_D',
  'i_l_1314_Q',
  'v_12_D',
  'v_12_Q',
  'v_13_D',
  'v_13_Q',
  

In [13]:
u_run_dict

{'ina': 0}